In [248]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as sts
from sklearn.pipeline import make_pipeline


sns.set()

In [249]:
!pip install openpyxl

In [250]:
df=pd.read_excel('../input/anz-transactions-dataset/ANZ synthesised transaction dataset.xlsx', engine="openpyxl")
df.head()

,status,card_present_flag,bpay_biller_code,account,currency,long_lat,txn_description,merchant_id,merchant_code,first_name,...,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,country,customer_id,merchant_long_lat,movement
0,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,POS,81c48296-73be-44a7-befa-d053f48ce7cd,NaN,Diana,...,26,Ashmore,QLD,2018-08-01T01:01:15.000+0000,16.25,a623070bfead4541a6b0fff8a09e706c,Australia,CUS-2487424745,153.38 -27.99,debit
1,authorized,0.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,830a451c-316e-4a6a-bf25-e37caedca49e,NaN,Diana,...,26,Sydney,NSW,2018-08-01T01:13:45.000+0000,14.19,13270a2a902145da9db4c951e04b51b9,Australia,CUS-2487424745,151.21 -33.87,debit
2,authorized,1.0,NaN,ACC-1222300524,AUD,151.23 -33.94,POS,835c231d-8cdf-4e96-859d-e9d571760cf0,NaN,Michael,...,38,Sydney,NSW,2018-08-01T01:26:15.000+0000,6.42,feb79e7ecd7048a5a36ec889d1a94270,Australia,CUS-2142601169,151.21 -33.87,debit
3,authorized,1.0,NaN,ACC-1037050564,AUD,153.10 -27.66,SALES-POS,48514682-c78a-4a88-b0da-2d6302e64673,NaN,Rhonda,...,40,Buderim,QLD,2018-08-01T01:38:45.000+0000,40.90,2698170da3704fd981b15e64a006079e,Australia,CUS-1614226872,153.05 -26.68,debit
4,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,b4e02c10-0852-4273-b8fd-7b3395e32eb0,NaN,Diana,...,26,Mermaid Beach,QLD,2018-08-01T01:51:15.000+0000,3.25,329adf79878c4cf0aeb4188b4691c266,Australia,CUS-2487424745,153.44 -28.06,debit


In [251]:
df.shape

(12043, 23)

In [252]:
df.columns

Index(['status', 'card_present_flag', 'bpay_biller_code', 'account',
       'currency', 'long_lat', 'txn_description', 'merchant_id',
       'merchant_code', 'first_name', 'balance', 'date', 'gender', 'age',
       'merchant_suburb', 'merchant_state', 'extraction', 'amount',
       'transaction_id', 'country', 'customer_id', 'merchant_long_lat',
       'movement'],
      dtype='object')

In [253]:
df.describe()

,card_present_flag,merchant_code,balance,age,amount
count,7717.000000,883.0,12043.000000,12043.000000,12043.000000
mean,0.802644,0.0,14704.195553,30.582330,187.933588
std,0.398029,0.0,31503.722652,10.046343,592.599934
min,0.000000,0.0,0.240000,18.000000,0.100000
25%,1.000000,0.0,3158.585000,22.000000,16.000000
50%,1.000000,0.0,6432.010000,28.000000,29.000000
75%,1.000000,0.0,12465.945000,38.000000,53.655000
max,1.000000,0.0,267128.520000,78.000000,8835.980000


In [254]:
df.customer_id.duplicated().sum()

11943

### there are many duplicate customer id's. so we gonna create a new data frame with unique id

In [255]:
salaries_df=pd.DataFrame({"Customer_id":df.customer_id.unique()})
salaries_df.head()

,Customer_id
0,CUS-2487424745
1,CUS-2142601169
2,CUS-1614226872
3,CUS-2688605418
4,CUS-4123612273


#### Now we'll try to calculate the salary for the each of the customers

In [256]:
df_freq=[]
df_amount=[]

for customer in range(len(salaries_df)):
    salary=df.loc[(df.customer_id==salaries_df.Customer_id[customer]) & (df.txn_description=='PAY/SALARY'),['date','amount']].groupby('date',as_index=False).sum()
    count=len(salary)
    if(count==0):
        df_freq.append(np.nan)
        df_amount.append(np.nan)
    else:
        dbp=[]
        for date in range(len(salary)-1):
            dbp.append((salary.date[date+1]- salary.date[date]).days)
        df_freq.append(max(dbp))
        df_amount.append(sts.mode(salary.amount))

salaries_df['sal_freq']=df_freq
salaries_df['amount']=df_amount
salaries_df['annual_salary']=salaries_df.amount/salaries_df.sal_freq*365

In [257]:
salaries_df.head()

,Customer_id,sal_freq,amount,annual_salary
0,CUS-2487424745,7,1013.67,52855.650000
1,CUS-2142601169,7,1002.13,52253.921429
2,CUS-1614226872,7,892.09,46516.121429
3,CUS-2688605418,14,2320.30,60493.535714
4,CUS-4123612273,7,1068.04,55690.657143


### From the above dataset we can also derive various information like weekly transactions,max_amount transaction,average transactions,median_balances etc

In [258]:
unique_id=salaries_df.Customer_id.unique()
unique_id[:10]

array(['CUS-2487424745', 'CUS-2142601169', 'CUS-1614226872',
       'CUS-2688605418', 'CUS-4123612273', 'CUS-3026014945',
       'CUS-2031327464', 'CUS-2317998716', 'CUS-1462656821',
       'CUS-3142625864'], dtype=object)

### Maximum Transactions

In [259]:
max_tran=[]
for i in unique_id:
    p=df.loc[df.customer_id==i,"amount"]
    max_tran.append(max(p))

In [260]:
max_tran[:5]

[1452.21, 2349.55, 892.09, 2320.3, 1068.04]

### Weekly Transactions

In [261]:
week_tran=[]

for i in unique_id:
    p=df.loc[df.customer_id==i,"date"]
    week_tran.append(len(p)/p.nunique()*7)


In [262]:
print(p.nunique())

60


#### Avergae Transactions

In [263]:
avg_tran=[]

for i in unique_id:
    p=df.loc[df.customer_id==i,"amount"]
    avg_tran.append(np.mean(p))

### Mode Transactions

In [264]:
mode_tran=[]

for i in unique_id:
    p=df.loc[df.customer_id==i,"amount"]
    mode_tran.append(p.median())

#### Age

In [265]:
age=[]

for i in unique_id:
    p=df.loc[df.customer_id==i,"age"]
    age.append(sts.mode(p))

### Gender

In [266]:
gender=[]

for i in unique_id:
    p=df.loc[df.customer_id==i,"gender"]
    gender.append(sts.mode(p))

### Merging feature into a single dataframe

In [267]:
feature_df=pd.DataFrame({
    
    'customer_id':unique_id,
    'gender':gender,
    'age':age,
    'mode_transaction':mode_tran,
    'average_transaction':avg_tran,
    'weekly_transaction':week_tran  
    
})

In [268]:
feature_df.head()

,customer_id,gender,age,mode_transaction,average_transaction,weekly_transaction
0,CUS-2487424745,F,26,11.28,45.348772,47.600000
1,CUS-2142601169,M,38,15.97,78.206106,28.662162
2,CUS-1614226872,F,40,25.38,74.465019,23.855263
3,CUS-2688605418,M,20,24.00,159.304186,14.333333
4,CUS-4123612273,F,43,29.46,166.508358,21.318182


### Merging the Annual_salary and the feature dataframe

In [269]:
data=pd.concat([feature_df,salaries_df.annual_salary],axis=1)

In [270]:
data.drop('customer_id',axis=1,inplace=True)

In [271]:
data.head()

,gender,age,mode_transaction,average_transaction,weekly_transaction,annual_salary
0,F,26,11.28,45.348772,47.600000,52855.650000
1,M,38,15.97,78.206106,28.662162,52253.921429
2,F,40,25.38,74.465019,23.855263,46516.121429
3,M,20,24.00,159.304186,14.333333,60493.535714
4,F,43,29.46,166.508358,21.318182,55690.657143


In [272]:
df_final=pd.get_dummies(data)
df_final.head()

,age,mode_transaction,average_transaction,weekly_transaction,annual_salary,gender_F,gender_M
0,26,11.28,45.348772,47.600000,52855.650000,1,0
1,38,15.97,78.206106,28.662162,52253.921429,0,1
2,40,25.38,74.465019,23.855263,46516.121429,1,0
3,20,24.00,159.304186,14.333333,60493.535714,0,1
4,43,29.46,166.508358,21.318182,55690.657143,1,0


#### Performing Standarization

In [273]:
from sklearn.preprocessing import StandardScaler
cols=['age','mode_transaction','average_transaction','weekly_transaction','annual_salary']

In [274]:
scaler=StandardScaler()

In [275]:
# df_final[cols]=scaler.fit_transform(df_final[cols])

In [276]:
df_final.head()

,age,mode_transaction,average_transaction,weekly_transaction,annual_salary,gender_F,gender_M
0,26,11.28,45.348772,47.600000,52855.650000,1,0
1,38,15.97,78.206106,28.662162,52253.921429,0,1
2,40,25.38,74.465019,23.855263,46516.121429,1,0
3,20,24.00,159.304186,14.333333,60493.535714,0,1
4,43,29.46,166.508358,21.318182,55690.657143,1,0


In [277]:
target=df_final['annual_salary']
input=df_final.drop(['annual_salary'],axis=1)

In [278]:
input

,age,mode_transaction,average_transaction,weekly_transaction,gender_F,gender_M
0,26,11.28,45.348772,47.600000,1,0
1,38,15.97,78.206106,28.662162,0,1
2,40,25.38,74.465019,23.855263,1,0
3,20,24.00,159.304186,14.333333,0,1
4,43,29.46,166.508358,21.318182,1,0
...,...,...,...,...,...,...
95,40,38.83,291.293333,14.000000,1,0
96,42,78.15,448.658413,11.307692,0,1
97,42,45.00,355.400250,8.484848,0,1
98,34,46.00,245.600169,15.296296,1,0


#### Train and Test split using sklearn

In [279]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(input,target,test_size=0.2)

### Linear Regression Model

In [280]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()

In [281]:
lr.fit(x_train,y_train)

LinearRegression()

In [282]:
lr.score(x_test,y_test)

0.5200929244831389

### RMSE

In [283]:
from sklearn.metrics import mean_squared_error as MSE

pred = lr.predict(x_test)
  
# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))

RMSE :  18634.216612


### R2 Score

In [284]:
from sklearn.metrics import r2_score
k=r2_score(y_test,pred)
k

0.5200929244831389